In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['SERS']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [9]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 0]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [10]:
list(zip(longforms, counts))

[('surface enhanced raman scattering', 546),
 ('surface enhanced raman spectroscopy', 264),
 ('surface enhanced raman spectroscopic', 4),
 ('surface enhanced raman', 4),
 ('surface enhanced raman scattering6', 1),
 ('surface enhanced raman spectra', 1),
 ('scattering spectra', 1),
 ('scattering spectroscopy', 1),
 ('surface enhanced raman spectroscopy17 20', 1),
 ('surface enhanced raman spectroscopy3 4', 1),
 ('surface enhanced resonance raman scattering', 1),
 ('self esteem rate scale short form', 1),
 ('stronger raman', 1),
 ('surface raman scattering', 1),
 ('raman cross section', 1),
 ('thus enabled portable and remote', 1),
 ('stripes ito slab to model far field', 1),
 ('enhanced of the raman active in the visible range', 1)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-04 16:07:43] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'surface enhanced raman': 'MESH:D013059',
  'surface enhanced raman scattering': 'MESH:D013059',
  'surface enhanced raman spectroscopic': 'MESH:D013059',
  'surface enhanced raman spectroscopy': 'MESH:D013059'},
 {'MESH:D013059': 'Spectrum Analysis, Raman'},
 ['MESH:D013059']]

In [55]:
grounding_map, names, pos_labels = [{'surface enhanced raman': 'MESH:D013059',
  'surface enhanced raman scattering': 'MESH:D013059',
  'surface enhanced raman spectroscopic': 'MESH:D013059',
  'surface enhanced raman spectroscopy': 'MESH:D013059'},
 {'MESH:D013059': 'Spectrum Analysis, Raman'},
 ['MESH:D013059']]

In [56]:
excluded_longforms = []

In [57]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [58]:
additional_entities = {'HGNC:10537': ['SARS1', ['SERS', 'SARS1', 'serine tRNA', 'seryl-tRNA']]}

In [59]:
unambiguous_agent_texts = {}

In [60]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [61]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [62]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [63]:
intersection1

[('HGNC:10537', 'HGNC:10537', 58)]

In [64]:
intersection2

[('MESH:D013059', 'HGNC:10537', 0)]

In [65]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [66]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [67]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-30 03:58:49] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-30 03:58:54] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 1.0 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [68]:
classifier.stats

{'label_distribution': {'MESH:D013059': 545, 'HGNC:10537': 17},
 'f1': {'mean': 1.0, 'std': 0.0},
 'precision': {'mean': 1.0, 'std': 0.0},
 'recall': {'mean': 1.0, 'std': 0.0},
 'MESH:D013059': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'HGNC:10537': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 1.0, 'std': 0.0}}}

In [69]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [70]:
disamb.dump(model_name, results_path)

In [71]:
print(disamb.info())

Disambiguation model for SERS

Produces the disambiguations:
	SARS1*	HGNC:10537
	Spectrum Analysis, Raman*	MESH:D013059

Class level metrics:
--------------------
Grounding               	Count	F1     
Spectrum Analysis, Raman*	545	    1.0
                   SARS1*	 17	    1.0

Weighted Metrics:
-----------------
	F1 score:	1.0
	Precision:	1.0
	Recall:		1.0

* Positive labels
See Docstring for explanation



In [75]:
model_to_s3(disamb)

In [72]:
preds = disamb.disambiguate(all_texts.values())

In [73]:
texts = [text for pred, text in zip(all_texts.values(), preds) if pred[0].startswith('HGNC')]

In [74]:
texts

[]